# Thesis work--2
# A new method for multivariate regression problem for improving fitness in Genetic Programming

## Equations

# Visulaiztions

### Plots for Equation 0 with different methods
![](images/Standard GP Equation 0 Total Runs 30.png?raw=true)
![](images/PCA GP Equation 0 Total Runs 30.png?raw=true)
![](images/DV GP Equation 0 Total Runs 30.png?raw=true)

### Plots for Equation 1 with different methods
![](images/Standard GP Equation 1 Total Runs 30.png?raw=true)
![](images/PCA GP Equation 1 Total Runs 30.png?raw=true)
![](images/DV GP Equation 1 Total Runs 30.png?raw=true)

### Plots for Equation 2 with different methods
![](images/Standard GP Equation 2 Total Runs 30.png?raw=true)
![](images/PCA GP Equation 2 Total Runs 30.png?raw=true)
![](images/DV GP Equation 2 Total Runs 30.png?raw=true)

### Plots for Equation 3 with different methods
![](images/Standard GP Equation  3 30.png?raw=true)
![](images/PCA GP Equation  3 30.png?raw=true)
![](images/DV GP Equation  3 30.png?raw=true)

### Plots for Equation 4 with different methods
![](images/Standard GP Equation 4 Total Runs 30.png?raw=true)
![](images/PCA GP Equation 4 Total Runs 30.png?raw=true)
![](images/DV GP Equation 4 Total Runs 30.png?raw=true)

### Plots for Equation 5 with different methods
![](images/Standard GP Equation 5 Total Runs 30.png?raw=true)
![](images/PCA GP Equation 5 Total Runs 30.png?raw=true)
![](images/DV GP Equation 5 Total Runs 30.png?raw=true)